# Random dead ends

### Based on features from Omnidian database 101, we compare K-Nearest Neighbors, Gradient Boosting, Random Forest, Bagging, and Logistic Regression.

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
import pickle

Get Data

Note to self: handle those duplicates!

In [9]:
data101 = pd.read_csv('data/eda101.csv', index_col='Ticket_Id')
data101.head()

,Asset_Id,Root_Cause,Ticket_Creation_Reason,asset_type,latitude,longitude,tilt,azimuth,Ticket_Origin,Date_Ticket_Initially_Assigned,Date_Ticket_Closed,installed_by,installation_date
Ticket_Id,,,,,,,,,,,,,
23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
23947,101112604,root_cause_normal_wear_and_tear,communication offline,Residential Solar PV,38.944642,-121.248833,37.0,190.0,origin_omnidian_customer,2018-10-22 17:25:27,2018-11-29,Williams Lifetime Builders Inc. DBA Lifetime S...,2013-04-04 00:00:00
27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,180.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00
27384,101113056,root_cause_normal_wear_and_tear,system inspection,Residential Solar PV,34.101697,-118.146646,18.0,90.0,origin_homeowner,2018-12-20 06:49:02,2019-02-09,Green Tech Solutions Inc.,2014-04-08 00:00:00


In [10]:
data101.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 825 entries, 23947 to 32514
Data columns (total 13 columns):
Asset_Id                          825 non-null int64
Root_Cause                        825 non-null object
Ticket_Creation_Reason            825 non-null object
asset_type                        825 non-null object
latitude                          825 non-null float64
longitude                         825 non-null float64
tilt                              825 non-null float64
azimuth                           825 non-null float64
Ticket_Origin                     825 non-null object
Date_Ticket_Initially_Assigned    825 non-null object
Date_Ticket_Closed                825 non-null object
installed_by                      825 non-null object
installation_date                 825 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 90.2+ KB


Assign target to Root_Cause and Train-Test-Split

Consider stratifying when train-test-split

In [11]:
X = data101.drop('Root_Cause', axis=1).copy()
y = data101['Root_Cause']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

One hot encoding: since our data is largely categorical but our models require numeric inputs, we one hot encode it.


In [18]:
categoricals = ['Asset_Id', 'Ticket_Creation_Reason', 'asset_type', 
                'Ticket_Origin', 'installed_by']

In [19]:
data101.columns

Index(['Asset_Id', 'Root_Cause', 'Ticket_Creation_Reason', 'asset_type',
       'latitude', 'longitude', 'tilt', 'azimuth', 'Ticket_Origin',
       'Date_Ticket_Initially_Assigned', 'Date_Ticket_Closed', 'installed_by',
       'installation_date'],
      dtype='object')

In [21]:
encoder = OneHotEncoder(categorical_features='categoricals', handle_unknown='ignore')
encoder.fit(X_train[categoricals])

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


IndexError: arrays used as indices must be of integer (or boolean) type

In [1]:
X_train = pd.DataFrame(encoder.transform(X_train).toarray,
                         columns=encoder.get_feature_names())

NameError: name 'pd' is not defined

In [27]:
X_train = pd.DataFrame(encoder.transform(X_train).toarray(), index_col='Ticket_Id',
                         columns=encoder.get_feature_names())
X_test = pd.DataFrame(encoder.transform(X_test).toarray(), index_col='Ticket_Id',
                        columns=encoder.get_feature_names())

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: could not convert string to float: 'Closed'

Training our models.

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
filename = '101_knn.pkl'
pickle.dump(knn, open(filename, 'wb'))

In [ ]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)

In [ ]:
filename = '101_lr.pkl'
pickle.dump(lr, open(filename, 'wb'))

In [ ]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train, y_train)

In [ ]:
filename = '101_gb.pkl'
pickle.dump(gb, open(filename, 'wb'))

In [ ]:
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=30)
dt.fit(X_train, y_train)

In [ ]:
filename = '101_dt.pkl'
pickle.dump(dt, open(filename, 'wb'))

In [ ]:
bg = BaggingClassifier(random_state=25565)
bg.fit(X_train, y_train)

In [ ]:
filename = '101_bg.pkl'
pickle.dump(bg, open(filename, 'wb'))

In [ ]:
rf = RandomForestClassifier(random_state=42, n_estimators=100)
rf.fit(X_train, y_train);

In [ ]:
filename = '101_rf.pkl'
pickle.dump(rf, open(filename, 'wb'))

Let's show our results

In [ ]:
knn.score(X_test, y_test)

In [ ]:
lr.score(X_test, y_test)

In [ ]:
gb.score(X_test, y_test)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
bg.score(X_test, y_test)

In [ ]:
rf.score(X_test, y_test)

## Build a function to take in Ticket_Id and return root cause probabilities.

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
y_test.loc[y_test.index == 13043]

In [ ]:
rf.predict_proba(X_test)

In [ ]:
rf.predict(X_test)[0:5]

In [ ]:
l_props = lr.predict_proba(X_test)[1]

In [ ]:
zipp = list(zip(lr.classes_, l_props))

In [ ]:
zipp

In [ ]:
#want a dictionary that gives probability for each class, 
#prediction, and ground truth
def display_preds_truth(model, Ticket_Id, X_test, y_test):
    probs = model.predict_proba(X_test.loc[X_test.index == Ticket_Id])
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test.loc[X_test.index == Ticket_Id])
    display['ground truth'] = y_test.loc[y_test.index == Ticket_Id]
    return display
        
    

In [ ]:
lr.predict_proba(X_test[8541])

In [ ]:
lr.predict_proba(X_test.loc[X_test.index == 8541])

In [ ]:
display_preds_truth(lr, 8541, X_test, y_test)

In [ ]:
#want a dictionary that gives probability for each class,
def display_probas(model, obs, X_test):
    probs = model.predict_proba(X_test)[obs]
    classes = model.classes_
    display = dict(zip(classes, probs))
    display['prediction'] = model.predict(X_test)[obs]
    return display

In [ ]:
display_probas(lr, 1, X_test)

In [ ]:
X_test.info()

In [ ]:
y_test